In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [6]:
from models.utils import load_model
# from models.contentbase import ContentBased
# from models.als import load_data
from models.dataset import Dataset

In [3]:
model = load_model("models/als.pkl")

In [7]:
def load_data(path):
    df = pd.read_csv(path, index_col=0)
    return Dataset(df, user="UserID", item="MovieID", rating="Rating")

In [8]:
ds = load_data("./data/rating.csv")

In [13]:
%%time
model.recommend(0, ds.get_csr()[0], N=20)

CPU times: total: 93.8 ms
Wall time: 73.7 ms


(array([ 390,  105,  167,  609,  104,  596,  207,  280,  632,  127,  273,
         124,  381,  128,  612,  588,   92, 1046,   85,  682]),
 array([1.088964  , 0.8012306 , 0.76091135, 0.7478144 , 0.74544036,
        0.742708  , 0.7148326 , 0.69618624, 0.68975836, 0.6547731 ,
        0.64351046, 0.64081377, 0.64046144, 0.6374482 , 0.63640064,
        0.61980534, 0.6041487 , 0.58303964, 0.5774966 , 0.5744972 ],
       dtype=float32))

In [16]:
def get_user_history(user_id):
    if user_id not in ds.user_list:
        raise ValueError(f"User id {user_id} not in dataset")
    return ds.get_user_history(user_id)

In [20]:
from sqlalchemy import create_engine

In [21]:
engine = create_engine("sqlite:///metadata")

In [56]:
def read_data(query, engine):
    return pd.read_sql_query(query, engine)
def get_movie_information(iid, table="movielens", sql_engine=engine):
    query = f"""SELECT * FROM {table} WHERE MovieID = {iid}"""
    return read_data(query, engine)

In [58]:
def get_metadata(*items):
    res = []
    for item in items[0]:
        res.append(get_movie_information(item))
    return pd.concat(res, ignore_index=True)

In [70]:
get_metadata(model.recommend(0, ds.get_csr()[0], N=20)[0])

,MovieID,Title,Genres,release_year
0,390,Faster Pussycat! Kill! Kill!,"Action,Comedy,Drama",1965
1,105,"Bridges of Madison County, The","Drama,Romance",1995
2,167,Feast of July,Drama,1995
3,609,Homeward Bound II: Lost in San Francisco,"Adventure,Children's",1996
4,104,Happy Gilmore,Comedy,1996
5,596,Pinocchio,"Animation,Children's",1940
6,207,"Walk in the Clouds, A","Drama,Romance",1995
7,280,Murder in the First,"Drama,Thriller",1995
8,632,Land and Freedom (Tierra y libertad),War,1995
9,127,"Silence of the Palace, The (Saimt el Qusur)",Drama,1994


In [71]:
def get_res_result(user_id, topk=10):
    if user_id not in ds.user_list:
        raise ValueError(f"User id {user_id} not in dataset")
    idx = np.where(ds.user_list == user_id)[0][0]
    item_id, score = model.recommend(idx, ds.get_csr()[idx], N=topk)
    df = get_metadata(item_id)
    df['score'] = score
    return df

In [76]:
# check assert
get_res_result(0)

ValueError: User id 0 not in dataset

In [74]:
get_res_result(2)

,MovieID,Title,Genres,release_year,score
0,264,"Enfer, L'",Drama,1994,0.995726
1,23,Assassins,Thriller,1995,0.981739
2,195,Something to Talk About,"Comedy,Drama,Romance",1995,0.967729
3,189,Reckless,Comedy,1995,0.956880
4,41,Richard III,"Drama,War",1995,0.917832
5,590,Dances with Wolves,"Adventure,Drama,Western",1990,0.852498
6,505,North,Comedy,1994,0.847792
7,765,Jack,"Comedy,Drama",1996,0.841336
8,280,Murder in the First,"Drama,Thriller",1995,0.808011
9,432,City Slickers II: The Legend of Curly's Gold,"Comedy,Western",1994,0.796700
